In [5]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# Scrap Team Lists

In [25]:
def get_wiki_url(html, year, soup):
    # Create a regular expression to match the target URLs.
    url_regex = rf'"http:\/\/{year}(.*)\.igem\.org\/(.*)"'
    # Find all matching URLs in the HTML.
    urls = re.findall(url_regex, html)
    if len(urls) == 0:
        # Create a regular expression to match the target URLs.
        url_regex = rf'"https:\/\/{year}(.*)\.igem\.org\/(.*)"'
        # Find all matching URLs in the HTML.
        urls = re.findall(url_regex, html)
        if len(urls) > 0:
            wiki = f"https://{year}{urls[0][0]}.igem.org/{urls[0][1]}"
            wiki = wiki.split('">Wiki<')[0]
        elif len(urls) == 0:
            # Create a regular expression to match the target URLs.
            url_regex = rf'"https:\/\/{year}\.igem\.wiki\/(.*)"'
            # Find all matching URLs in the HTML.
            urls = re.findall(url_regex, html)
            if len(urls) > 0:
                wiki = f"https://{year}.igem.wiki/{urls[0]}"
                wiki = wiki.split('">Wiki<')[0]
            else: 
                wiki = None
    else:
        wiki = f"https://{year}{urls[0][0]}.igem.org/{urls[0][1]}"
        wiki = wiki.split('">Wiki<')[0]
    # if soup.find('a', href=re.compile('wiki')) is not None:
    #     wiki = soup.find('a', href=re.compile('wiki')).get('href')
    # elif soup.find('a', href=re.compile('org')) is not None:
    #     wiki = soup.find('a', href=re.compile('igem')).get('href')
    # else:
    #     wiki = None
    return(wiki)
def extract_each_row(row, year):
    # Create a BeautifulSoup object
    soup = BeautifulSoup(row, 'html.parser')

    # Extract the team name
    team_name = soup.find('a', class_='team_name').get_text()
    link_team = soup.find('a', class_='team_name').get('href')
    # wiki_url = soup.find('a', href=True).get('href')
    wiki_url = get_wiki_url(str(row), year, soup)

    # extract other information
    infos = []
    for a in row.split('<td data-order='):
        infos.append(a.split('>')[-1])
    region = infos[2]
    country = infos[3]
    track = infos[4]
    kind = infos[5]
    section = infos[6]
    application = infos[7]
    return(team_name, link_team, wiki_url, region, country, track, kind, section, application)

In [26]:
teams = []
links = []
wikis = []
regions = []
countries = []
tracks = []
kinds = []
sections = []
apps = []
years = []
year_range = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


for year in year_range:
    url = 'https://old.igem.org/Team_List?year=' + str(year)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    all = re.split(r"<tr>", str(soup.table.tbody.tr))[1:]
    print(year)
    for i in range(len(all)):
        team_name, link_team, wiki_url, region, country, track, kind, section, application = extract_each_row(all[i], year)
        teams.append(team_name)
        links.append(link_team)
        wikis.append(wiki_url)
        regions.append(region)
        countries.append(country)
        tracks.append(track)
        kinds.append(kind)
        sections.append(section)
        apps.append(application)
        years.append(year)

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [27]:
# Create a DataFrame
df = pd.DataFrame({
    'Team name': teams,
    'Profile team': links,
    'Wiki URL': wikis,
    'Region': regions,
    'Country': countries,
    'Track': tracks,
    'Kind': kinds,
    'Section': sections,
    'Application': apps,
    'Year' : years
})


# Print the DataFrame
df

,Team name,Profile team,Wiki URL,Region,Country,Track,Kind,Section,Application,Year
0,Alberta_NINT,https://old.igem.org/Team.cgi?team_id=102,https://2008.igem.org/Team:Alberta_NINT,North America,Canada,,Collegiate,iGEM,Accepted,2008
1,Bay_Area_RSI,https://old.igem.org/Team.cgi?team_id=80,https://2008.igem.org/Team:Bay_Area_RSI,North America,United States,,Collegiate,iGEM,Accepted,2008
2,BCCS-Bristol,https://old.igem.org/Team.cgi?team_id=135,https://2008.igem.org/Team:BCCS-Bristol,Europe,United Kingdom,,Collegiate,iGEM,Accepted,2008
3,Beijing_Normal,https://old.igem.org/Team.cgi?team_id=78,https://2008.igem.org/Team:Beijing_Normal,Asia,China,,Collegiate,iGEM,Accepted,2008
4,Bologna,https://old.igem.org/Team.cgi?team_id=79,https://2008.igem.org/Team:Bologna,Europe,Italy,,Collegiate,iGEM,Accepted,2008
...,...,...,...,...,...,...,...,...,...,...
4227,Zhejiang-United,https://old.igem.org/Team.cgi?team_id=4876,https://2023.igem.wiki/zhejiang-united,Asia,China,High School,Commercial,hs,Accepted,2023
4228,ZJFH-Nanjing,https://old.igem.org/Team.cgi?team_id=4941,https://2023.igem.wiki/zjfh-nanjing,Asia,China,High School,Commercial,hs,Accepted,2023
4229,ZJU-China,https://old.igem.org/Team.cgi?team_id=4628,https://2023.igem.wiki/zju-china,Asia,China,Agriculture,Collegiate,under,Accepted,2023
4230,ZJUintl-China,https://old.igem.org/Team.cgi?team_id=4776,https://2023.igem.wiki/zjuintl-china,Asia,China,Therapeutics,Collegiate,under,Accepted,2023


In [28]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4232 entries, 0 to 4231
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Team name     4232 non-null   object
 1   Profile team  4232 non-null   object
 2   Wiki URL      4232 non-null   object
 3   Region        4232 non-null   object
 4   Country       4232 non-null   object
 5   Track         4232 non-null   object
 6   Kind          4232 non-null   object
 7   Section       4232 non-null   object
 8   Application   4232 non-null   object
 9   Year          4232 non-null   int64 
dtypes: int64(1), object(9)
memory usage: 330.8+ KB


In [34]:
# Print the selected rows
len(df[df.isnull().any(axis=1)])

0

In [30]:
df.to_csv('team_lists_all_years.csv', index=False)

# Scrap Team Profile

In [85]:
url = 'https://old.igem.org/Team.cgi?team_id=102' #2008
# url = 'https://old.igem.org/Team.cgi?team_id=385' #2010
# url = 'https://old.igem.org/Team.cgi?team_id=1161' #2013
# url = 'https://old.igem.org/Team.cgi?team_id=2815' #2018
# url = 'https://old.igem.org/Team.cgi?team_id=3743' # 2021
# url = 'https://old.igem.org/Team.cgi?team_id=4769' #2023

In [83]:
def get_title_abstract(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    content = soup.findAll(id = 'table_abstract')[0].tr.td.get_text().split('\n')
    title = content[0]
    abstract = " ".join(content[1:])
    return(title, abstract)

In [86]:
get_title_abstract(url)

('-- Not provided yet --', '-- No abstract provided yet -- ')

In [87]:
titles = []
abstracts = []
for i in df['Profile team']:
    title, abs = get_title_abstract(i)
    titles.append(title)
    abstracts.append(abs)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

# Scrap Wiki

In [6]:
url = 'https://2022.igem.wiki/tu-eindhoven/engineering'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Required meta tags -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://static.igem.wiki/common/icons/favicons/igem-2022.svg" rel="shortcut icon"/>
  <link href="https://creativecommons.org/licenses/by/4.0/" rel="license"/>
  <!-- Bootstrap CSS -->
  <link href="static/bootstrap.min.css" rel="stylesheet"/>
  <!-- Custom CSS -->
  <link href="static/style.css" rel="stylesheet"/>
  <title>
   Engineering Success | TU-Eindhoven - iGEM 2022
  </title>
 </head>
 <body>
  <!-- Navigation -->
  <nav class="navbar navbar-expand-lg navbar-dark fixed-top" style="background-color: #1b263b;">
   <div class="container">
    <!---- TEAM NAME ---->
    <a class="navbar-brand" href="https://2022.igem.wiki/tu-eindhoven/" style="max-width: 79%">
     <img class="w-25" src="https://static.igem.wiki/teams/4160/wiki/footer/mpact-wit.svg" style="padding: 5px"/>
     TU-Eindhoven
    </a>
  